In [3]:
import numpy as np
import pandas as pd
import requests
import os
import json

In [4]:
congressapi = os.getenv('congressapi')
opensecretsapi = os.getenv('opensecretsapi')

## Congress Bills API for the 118th Congress

In [5]:
root = 'https://api.congress.gov/v3'
congress = '118'
endpoint = f'/bill/{congress}'

In [6]:
parameters = {'format': 'json',
             'offset': 0,
             'limit': 250,
             'sort': 'updateDate+desc',
             'api_key': congressapi}

In [7]:
r = requests.get('http://httpbin.org/user-agent')
useragent = json.loads(r.text)['user-agent']

In [8]:
headers = {'User-Agent': useragent,
          'From': 'jtb3sud@virginia.edu'}

In [9]:
r = requests.get(root + endpoint, params = parameters, headers=headers)

In [10]:
r

<Response [200]>

In [11]:
myjson = json.loads(r.text)

In [12]:
myjson['bills'][3]['number']

'375'

In [13]:
with open('bills.json', 'w') as f:
    json.dump(myjson, f)

In [14]:
#[x['number'] for x in myjson['bills']]

In [15]:
billsdf = pd.json_normalize(myjson, record_path = ['bills'])

In [16]:
billsdf

,congress,number,originChamber,originChamberCode,title,type,updateDate,updateDateIncludingText,url,latestAction.actionDate,latestAction.actionTime,latestAction.text
0,118,4367,House,H,Department of Homeland Security Appropriations...,HR,2023-09-28,2023-09-28T19:26:13Z,https://api.congress.gov/v3/bill/118/hr/4367?f...,2023-09-28,00:48:14,Committee of the Whole House on the state of t...
1,118,730,House,H,Providing for consideration of the bill (H.R. ...,HRES,2023-09-28,2023-09-28T19:30:46Z,https://api.congress.gov/v3/bill/118/hres/730?...,2023-09-28,NaN,"Placed on the House Calendar, Calendar No. 39."
2,118,377,Senate,S,A resolution recognizing Hispanic Restaurant W...,SRES,2023-09-28,2023-09-28T19:28:42Z,https://api.congress.gov/v3/bill/118/sres/377?...,2023-09-27,NaN,"Submitted in the Senate, considered, and agree..."
3,118,375,Senate,S,A resolution supporting the designation of Sep...,SRES,2023-09-28,2023-09-28T19:28:42Z,https://api.congress.gov/v3/bill/118/sres/375?...,2023-09-27,NaN,"Submitted in the Senate, considered, and agree..."
4,118,374,Senate,S,"A resolution designating September 2023 as ""Na...",SRES,2023-09-28,2023-09-28T19:28:42Z,https://api.congress.gov/v3/bill/118/sres/374?...,2023-09-27,NaN,"Submitted in the Senate, considered, and agree..."
...,...,...,...,...,...,...,...,...,...,...,...,...
245,118,5157,House,H,To amend the Cooperative Forestry Assistance A...,HR,2023-09-26,2023-09-26T07:13:59Z,https://api.congress.gov/v3/bill/118/hr/5157?f...,2023-09-25,NaN,Referred to the Subcommittee on Forestry.
246,118,5015,House,H,Seedlings for Sustainable Habitat Restoration ...,HR,2023-09-26,2023-09-26T07:13:59Z,https://api.congress.gov/v3/bill/118/hr/5015?f...,2023-09-25,NaN,Referred to the Subcommittee on Forestry.
247,118,4943,House,H,Enabling Farmers to Benefit from Processing Nu...,HR,2023-09-26,2023-09-26T07:13:59Z,https://api.congress.gov/v3/bill/118/hr/4943?f...,2023-09-25,NaN,"Referred to the Subcommittee on Nutrition, For..."
248,118,4920,House,H,Responsible Wildland Fire Recovery Act,HR,2023-09-26,2023-09-26T07:13:58Z,https://api.congress.gov/v3/bill/118/hr/4920?f...,2023-09-25,NaN,Referred to the Subcommittee on Forestry.


In [17]:
root = 'http://www.opensecrets.org/api/'
parameters = {"method": 'getLegislators',
             'apikey': opensecretsapi,
             'id': 'VA',
             'output': 'json'}

In [18]:
r = requests.get(root, params=parameters, headers=headers)

In [19]:
r

<Response [200]>

In [20]:
myjson = json.loads(r.text)

In [21]:
#myjson

In [46]:
root = 'https://api.congress.gov/v3'
congress = '118'
bill_type = 'hr'
endpoint = f'/bill/{congress}/{bill_type}'

house_bills = []
for i in range(0,1001, 250):

    parameters = {'format': 'json',
                 'offset': i,
                 'limit': 250,
                 'sort': 'updateDate+desc',
                 'api_key': congressapi}
    
    r = requests.get(root + endpoint, params = parameters, headers=headers)
    myjson = json.loads(r.text)
    bills = myjson['bills']
    house_bills.append(bills)

In [47]:
len(house_bills)

5

In [48]:
list_of_dfs = []
for i in range(4):
    list_of_dfs.append(pd.DataFrame.from_dict(house_bills[i]))

new_house_bills = pd.concat(list_of_dfs)

In [49]:
new_house_bills

,congress,latestAction,number,originChamber,originChamberCode,title,type,updateDate,updateDateIncludingText,url
0,118,"{'actionDate': '2023-09-28', 'actionTime': '00...",4367,House,H,Department of Homeland Security Appropriations...,HR,2023-09-28,2023-09-28T19:26:13Z,https://api.congress.gov/v3/bill/118/hr/4367?f...
1,118,"{'actionDate': '2023-09-27', 'actionTime': '18...",4365,House,H,"Department of Defense Appropriations Act, 2024",HR,2023-09-28,2023-09-28T19:26:13Z,https://api.congress.gov/v3/bill/118/hr/4365?f...
2,118,"{'actionDate': '2023-09-27', 'actionTime': '17...",4368,House,H,"Agriculture, Rural Development, Food and Drug ...",HR,2023-09-28,2023-09-28T18:31:17Z,https://api.congress.gov/v3/bill/118/hr/4368?f...
3,118,"{'actionDate': '2023-09-27', 'text': 'Received...",5110,House,H,Protecting Hunting Heritage and Education Act,HR,2023-09-28,2023-09-28T19:30:45Z,https://api.congress.gov/v3/bill/118/hr/5110?f...
4,118,"{'actionDate': '2023-09-27', 'text': 'Motion t...",3935,House,H,Securing Growth and Robust Leadership in Ameri...,HR,2023-09-28,2023-09-28T19:13:41Z,https://api.congress.gov/v3/bill/118/hr/3935?f...
...,...,...,...,...,...,...,...,...,...,...
245,118,"{'actionDate': '2023-07-27', 'text': 'Referred...",5028,House,H,Family Poverty is Not Child Neglect Act,HR,2023-09-22,2023-09-22T13:13:21Z,https://api.congress.gov/v3/bill/118/hr/5028?f...
246,118,"{'actionDate': '2023-07-27', 'text': 'Referred...",5009,House,H,WILD Act,HR,2023-09-22,2023-09-22T13:13:19Z,https://api.congress.gov/v3/bill/118/hr/5009?f...
247,118,"{'actionDate': '2023-07-27', 'text': 'Referred...",5011,House,H,Promotion and Expansion of Private Employee Ow...,HR,2023-09-26,2023-09-26T17:45:52Z,https://api.congress.gov/v3/bill/118/hr/5011?f...
248,118,"{'actionDate': '2023-07-27', 'text': 'Referred...",5001,House,H,Special Advisors for Insular Areas Act,HR,2023-09-22,2023-09-22T13:13:19Z,https://api.congress.gov/v3/bill/118/hr/5001?f...


In [56]:
root = 'https://api.congress.gov/v3'
congress = '118'
bill_type = 's'
endpoint = f'/bill/{congress}/{bill_type}'

house_bills = []
for i in range(0,1001, 250):

    parameters = {'format': 'json',
                 'offset': i,
                 'limit': 250,
                 'sort': 'updateDate+desc',
                 'api_key': congressapi}
    
    r = requests.get(root + endpoint, params = parameters, headers=headers)
    myjson = json.loads(r.text)
    bills = myjson['bills']
    house_bills.append(bills)

list_of_dfs = []
for i in range(4):
    list_of_dfs.append(pd.DataFrame.from_dict(house_bills[i]))

senate_bills = pd.concat(list_of_dfs)

senate_bills

,congress,latestAction,number,originChamber,originChamberCode,title,type,updateDate,updateDateIncludingText,url
0,118,"{'actionDate': '2023-09-27', 'text': 'Read twi...",2961,Senate,S,A bill to ensure greater equity in Federal dis...,S,2023-09-28,2023-09-28T04:16:00Z,https://api.congress.gov/v3/bill/118/s/2961?fo...
1,118,"{'actionDate': '2023-09-27', 'text': 'Read twi...",2960,Senate,S,"A bill to modify certain notice requirements, ...",S,2023-09-28,2023-09-28T04:16:00Z,https://api.congress.gov/v3/bill/118/s/2960?fo...
2,118,"{'actionDate': '2023-09-27', 'text': 'Placed o...",654,Senate,S,Delaware River Basin Conservation Reauthorizat...,S,2023-09-28,2023-09-28T04:14:08Z,https://api.congress.gov/v3/bill/118/s/654?for...
3,118,"{'actionDate': '2023-09-27', 'text': 'Placed o...",2959,Senate,S,An original bill to amend the Comprehensive En...,S,2023-09-28,2023-09-28T04:16:00Z,https://api.congress.gov/v3/bill/118/s/2959?fo...
4,118,"{'actionDate': '2023-09-27', 'text': 'Placed o...",2958,Senate,S,An original bill to amend the Coastal Barrier ...,S,2023-09-28,2023-09-28T04:16:00Z,https://api.congress.gov/v3/bill/118/s/2958?fo...
...,...,...,...,...,...,...,...,...,...,...
245,118,"{'actionDate': '2023-06-22', 'text': 'Read twi...",2134,Senate,S,Smoke Exposure Crop Insurance Act of 2023,S,2023-09-22,2023-09-22T12:40:24Z,https://api.congress.gov/v3/bill/118/s/2134?fo...
246,118,"{'actionDate': '2023-06-22', 'text': 'Read twi...",2135,Senate,S,Smoke Exposure Research Act of 2023,S,2023-09-22,2023-09-22T12:40:24Z,https://api.congress.gov/v3/bill/118/s/2135?fo...
247,118,"{'actionDate': '2023-06-22', 'text': 'Read twi...",2137,Senate,S,Preserving Access to Home Health Act of 2023,S,2023-09-22,2023-09-22T13:17:27Z,https://api.congress.gov/v3/bill/118/s/2137?fo...
248,118,"{'actionDate': '2023-06-22', 'text': 'Read twi...",2138,Senate,S,Workforce DATA Act,S,2023-09-22,2023-09-22T13:17:27Z,https://api.congress.gov/v3/bill/118/s/2138?fo...


In [65]:
root = 'https://api.congress.gov/v3'
congress = '118'
bill_type = 's'


cosponsors = []
for i in range(0,100):
    bill_number = senate_bills.iloc[i]['number']
    endpoint = f'/bill/{congress}/{bill_type}/{bill_number}/cosponsors'
    parameters = {'format': 'json',
                 'sort': 'updateDate+desc',
                 'api_key': congressapi}
    
    r = requests.get(root + endpoint, params = parameters, headers=headers)
    myjson = json.loads(r.text)
    cosponsor = myjson['cosponsors']
    cosponsors.append({"Bill Number": f"S {bill_number}", "Cosponsors": cosponsor})

In [66]:
cosponsors = pd.DataFrame.from_dict(cosponsors)
cosponsors

,Bill Number,Cosponsors
0,S 2961,"[{'bioguideId': 'B001288', 'firstName': 'Cory'..."
1,S 2960,"[{'bioguideId': 'S001203', 'firstName': 'Tina'..."
2,S 654,"[{'bioguideId': 'B001288', 'firstName': 'Cory'..."
3,S 2959,[]
4,S 2958,[]
...,...,...
95,S 2875,[]
96,S 1657,"[{'bioguideId': 'K000377', 'firstName': 'Mark'..."
97,S 2216,[]
98,S 1097,"[{'bioguideId': 'F000062', 'firstName': 'Diann..."


In [72]:
root = 'http://www.opensecrets.org/api/'
parameters = {"method": 'getLegislators',
             'apikey': opensecretsapi,
             'id': 'VA',
             'output': 'json'}

headers = {'User-Agent': useragent,
          'From': 'jtb3sud@virginia.edu'}

r = requests.get(root, params = parameters, headers=headers)
myjson = json.loads(r.text)




cids=[]
for i in range(0, len(myjson['response']['legislator'])):
    cid = myjson['response']['legislator'][i]['@attributes']['cid']
    cids.append(cid)

cids

['N00029459',
 'N00048315',
 'N00002147',
 'N00051961',
 'N00045557',
 'N00042296',
 'N00041418',
 'N00036018',
 'N00032029',
 'N00041002',
 'N00029891',
 'N00033177',
 'N00002097']

In [74]:
financial_information = []

for i in range(0, len(cids)):
    parameters = {"method": 'memPFDprofile',
             'apikey': opensecretsapi,
             'cid': cids[i],
             'output': 'json'}

    r = requests.get(root, params = parameters, headers=headers)
    myjson = json.loads(r.text)
    data = myjson['response']['member_profile']
    financial_information.append({'CID': cids[i], "Financial Information": data})
financial_information = pd.DataFrame.from_dict(financial_information)
financial_information
    

,CID,Financial Information
0,N00029459,"{'@attributes': {'name': 'Wittman, Rob', 'data..."
1,N00048315,"{'@attributes': {'name': '', 'data_year': '201..."
2,N00002147,"{'@attributes': {'name': 'Scott, Bobby', 'data..."
3,N00051961,"{'@attributes': {'name': '', 'data_year': '201..."
4,N00045557,"{'@attributes': {'name': '', 'data_year': '201..."
5,N00042296,"{'@attributes': {'name': '', 'data_year': '201..."
6,N00041418,"{'@attributes': {'name': '', 'data_year': '201..."
7,N00036018,"{'@attributes': {'name': '', 'data_year': '201..."
8,N00032029,"{'@attributes': {'name': 'Griffith, Morgan', '..."
9,N00041002,"{'@attributes': {'name': '', 'data_year': '201..."
